# 3. 시카고 샌드위치 맛집 분석

- 웹 스크래칭
- Beautiful Soup

- 시카고 샌드위치 맛집 리스트를 정리

In [ ]:
# 필요한 모듈 로드
import pandas as pd

from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen

## 3.1 시카고 샌드위치 맛집 소개 사이트 접근

In [ ]:
# 사이트 주소 정의
url_base = 'http://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

# 코렙으로 인한 문제 해결
req = Request(url,headers = {'User-Agent': 'Mozilla/5.0'})
html = urlopen(req)
soup = BeautifulSoup(html, "html.parser")

soup

In [ ]:
print(soup.find_all('div','sammy'))

In [ ]:
# 50가지의 샌드위치 가게 수 출력
len(soup.find_all('div', 'sammy'))

50

In [ ]:
# 첫 번째 열 생성
print(soup.find_all('div', 'sammy')[0])

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>


정상적으로 접근한것을 확인

## 3.2 접근한 웹 페이지에서 원하는 데이터 추출

In [ ]:
# 'div', 클래스 이름이 'sammy'인 데이터의 첫 행
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)

bs4.element.Tag

In [ ]:
# 클래스 이름을 이용하여 find
tmp_one.find(class_='sammyRank')

<div class="sammyRank">1</div>

In [ ]:
# 그 클래스에서 텍스트 데이터 추출
tmp_one.find(class_='sammyRank').get_text()

'1'

In [ ]:
# 텍스트 정보 추출
tmp_one.find(class_='sammyListing').get_text()

'BLT\nOld Oak Tap\nRead more '

In [ ]:
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [ ]:
# 정규식을 활용하여 텍스데이터 전처리
import re

tmp_string = tmp_one.find(class_='sammyListing').get_text()

re.split(('\n|\r\n'), tmp_string)

print(re.split(('\n|\r\n'), tmp_string)[0])
print(re.split(('\n|\r\n'), tmp_string)[1])

BLT
Old Oak Tap


In [ ]:
from urllib.parse import urljoin

In [ ]:
## 각 정보들 추출

rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()

    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    ## url 조합
    url_add.append(urljoin(url_base, item.find('a')['href']))

In [ ]:
rank[:5]

In [ ]:
main_menu[:5]

['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']

In [ ]:
cafe_name[:5]

['Old Oak Tap', 'Au Cheval', 'Xoco', 'Al’s Deli', 'Publican Quality Meats']

In [ ]:
url_add[:5]

['http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/']

In [ ]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

(50, 50, 50, 50)

In [ ]:
# 판다스 데이터 형식으로 변환
import pandas as pd

data = {'Rank':rank, 'Menu':main_menu, 'Cafe':cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [ ]:
# 데이터 저장
df.to_csv('/content/drive/MyDrive/스터디/데이터 주무르기/data/03_best_sandwiches_list_chicago.csv', sep=',', 
         index = False, encoding='UTF-8')


## 3.3 다수의 웹 페이지에 자동으로 접근해서 원하는 정보

In [ ]:
# 다수의 웹 페이지 주소를 활용
df['URL'][0]

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [ ]:
# 코렙으로 인한 문제 해결
req = Request(df['URL'][0],headers = {'User-Agent': 'Mozilla/5.0'})
html = urlopen(req)
soup_tmp = BeautifulSoup(html, "html.parser")
soup_tmp

In [ ]:
print(soup_tmp.find('p', 'addy'))

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>


In [ ]:
price_tmp = soup_tmp.find('p', 'addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [ ]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [ ]:
price_tmp.split()[0][:-1]

'$10'

In [ ]:
' '.join(price_tmp.split()[1:-2])

'2109 W. Chicago Ave.,'

In [ ]:
price = []
address = []

for n in df.index[:3]:
  # 코렙으로 인한 문제 해결
    req = Request(df['URL'][n],headers = {'User-Agent': 'Mozilla/5.0'})
    html= urlopen(req)
    soup_tmp = BeautifulSoup(html, "html.parser")
        
    gettings = soup_tmp.find('p', 'addy').get_text()
    
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [ ]:
price

['$10', '$9', '$9.50']

In [ ]:
address

['2109 W. Chicago Ave.,', '800 W. Randolph St.,', '445 N. Clark St.,']

## 3.4 tqdm 모듈 상태 진행바까지 적용

In [ ]:
from tqdm import tqdm_notebook

price = []
address = []

for n in tqdm_notebook(df.index):
  # 코렙으로 인한 문제 해결
    req = Request(df['URL'][n],headers = {'User-Agent': 'Mozilla/5.0'})
    html= urlopen(req)
    soup_tmp = BeautifulSoup(html, "html.parser")
        
    gettings = soup_tmp.find('p', 'addy').get_text()
    
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/50 [00:00<?, ?it/s]

### 50개 웹 페이지에 대한 정보 획득

In [ ]:
price

['$10',
 '$9',
 '$9.50',
 '$9.40',
 '$10',
 '$7.25',
 '$16',
 '$10',
 '$9',
 '$17',
 '$11',
 '$5.49',
 '$14',
 '$10',
 '$13',
 '$4.50',
 '$11.95',
 '$11.50',
 '$6.25',
 '$15',
 '$5',
 '$6',
 '$8',
 '$5.99',
 '$7.52',
 '$11.95',
 '$7.50',
 '$12.95',
 '$7',
 '$21',
 '$9.79',
 '$9.75',
 '$13',
 '$7.95',
 '$9',
 '$9',
 '$8',
 '$8',
 '$7',
 '$6',
 '$7.25',
 '$11',
 '$6',
 '$9',
 '$5.49',
 '$8',
 '$6.50',
 '$7.50',
 '$8.75',
 '$6.85']

In [ ]:
address

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,',
 '100 E. Walton',
 '1639 S. Wabash Ave.,',
 '2211 W. North Ave.,',
 '3619 W. North Ave.,',
 '3267 S. Halsted St.,',
 '2537 N. Kedzie Blvd.,',
 'Multiple',
 '3124 N. Broadway,',
 '3455 N. Southport Ave.,',
 '2657 N. Kedzie Ave.,',
 '1120 W. Grand Ave.,',
 '1141 S. Jefferson St.,',
 '333 E. Benton Pl.,',
 '1411 N. Wells St.,',
 '1747 N. Damen Ave.,',
 '3209 W. Irving Park',
 'Multiple',
 '5347 N. Clark St.,',
 '2954 W. Irving Park Rd.,',
 'Multiple',
 '191 Skokie Valley Rd., Highland Park,',
 'Multiple',
 '1818 W. Wilson Ave.,',
 '2517 W. Division St.,',
 '218 W. Kinzie',
 'Multiple',
 '1547 N. Wells St.,',
 '415 N. Milwaukee Ave.,',
 '1840 N. Damen Ave.,',
 '1220 W. Webster Ave.,',
 '5357 N. Ashland Ave.,',
 '1834 W. Montrose Ave.,',
 '615 N. State St.,',
 'Multiple',
 '241 N. York Rd., Elmhurst,',
 '1323 E. 57th St.,',
 '655 Forest Ave., Lake Forest,',
 'Hotel L

In [ ]:
df.head(10)

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,6,Belgian Chicken Curry Salad,Hendrickx Belgian Bread Crafter,https://www.chicagomag.com/Chicago-Magazine/No...
6,7,Lobster Roll,Acadia,http://www.chicagomag.com/Chicago-Magazine/Nov...
7,8,Smoked Salmon Salad,Birchwood Kitchen,http://www.chicagomag.com/Chicago-Magazine/Nov...
8,9,Atomica Cemitas,Cemitas Puebla,http://www.chicagomag.com/Chicago-Magazine/Nov...
9,10,Grilled Laughing Bird Shrimp and Fried Po’ Boy,Nana,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [ ]:
len(price), len(address), len(df)

(50, 50, 50)

In [ ]:
(50, 50, 50)
df['Price'] = price
df['Address'] = address

df = df.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df.set_index('Rank', inplace=True)
df.head()

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


## 3.5 맛집 위치를 지도에 표기하기

In [ ]:
!pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=a828c5ff0b0fc25aa85388f7d5ed5f148f201b164b25851bbea93064f0666ca5
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


In [ ]:
import folium
import pandas as pd
import googlemaps
import numpy as np

In [ ]:
gmaps_key = "****************" 
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
## 구글 맵을 활용하여 지역 데이터 불러오기
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
        
    else:
        lat.append(np.nan)
        lng.append(np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
len(lat), len(lng)

(50, 50)

In [ ]:
df['lat'] = lat
df['lng'] = lng
df.head()

,Cafe,Menu,Price,Address,lat,lng
Rank,,,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,",41.895558,-87.679967
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,",41.884639,-87.647590
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,",41.890523,-87.630783
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,",42.058322,-87.683748
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,",41.886604,-87.648536


In [ ]:
## 지도 시각화
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                     zoom_start=11)

for n in df.index:
    if df['Address'][n] != 'Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]], 
                                      popup=df['Cafe'][n]).add_to(mapping)

mapping

In [ ]:
mapping.save("/content/drive/MyDrive/스터디/데이터 주무르기/3장_시카고 샌드위치 맛집 위치.html")